In [1]:
import datetime
import time
import requests
import pandas as pd
import json
import os
import warnings
import requests
import numpy as np



def filter_location(accidents, counties_list):
    """_summary_

    Args:
        accidents (DataFrame): full API call from polisens API
        counties_list (str list): name of counties 

    Returns:
        DataFrame: location only of selected counties
    """
    locations = pd.DataFrame(columns=['name', 'gps'])
    locations['name'] = accidents['location'].apply(lambda x: x['name'])
    locations['gps'] = accidents['location'].apply(lambda x: x['gps'])

    # Filter the locations to only include those in Stockholm's counties
    stockholm_locations = locations[locations['name'].isin(counties_list)]
    return stockholm_locations


def choose_counties(counties: list):
    """counties eg = "01" to "25"  OBS "02" non existent"""

    with open("../../data/kommuner.json", "r") as f:
        kommuner = json.load(f)

    chosen_counties = []
    for county in counties:
        chosen_counties.append(kommuner[county])

    chosen_counties = [kommun for sublist in chosen_counties for kommun in sublist]

    return chosen_counties

In [2]:

def get_polisens_events(date):
    url = f"https://polisen.se/api/events?DateTime={date}"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for HTTP errors
        events = response.json()
        return events

    except requests.exceptions.RequestException as e:
        print("Error fetching data:", e)
        return None


def filter_event_type(events, type="Trafikolycka"):
    if events is None:
        return 0
    df = pd.DataFrame(events)
    filtered_events = df[df["type"].str.contains(type, na=False)]
    return filtered_events


def get_daily_events(date = None, type="Trafikolycka"):
    """Get events (accidents often) for a specific date

    Args:
        date ("YYYY-MM-DD" , optional): Defaults to todays date.
        type (str, optional): Type of report to count. Defaults to "Trafikolycka".

    Returns:
        int: Number of reports made on the specified date
    """
    
    if date is None:
        date = datetime.date.today()

    events = get_polisens_events(date)
    filtered_events = filter_event_type(events, type)
    
    return filtered_events

In [3]:
def update_csv_with_yesterdays_accidents(csv_path):
    # Read the existing CSV file
    df = pd.read_csv(csv_path)

    # Get yesterday's date
    yesterday = datetime.datetime.now() - datetime.timedelta(1)
    yesterday_str = yesterday.strftime('%Y-%m-%d')

    # Check if yesterday's data is already present
    if yesterday_str in df['date'].values:
        print(f"Data for {yesterday_str} is already present in the CSV.")
        return

    # Get the number of accidents for yesterday
    yesterdays_accidents = get_daily_events(date=yesterday_str, type="Trafikolycka")

    location_acc = filter_location(yesterdays_accidents, ["01", "03","04","17", "19", "20"])
    no_yesterdays_accidents = len(location_acc)

    # Create a new DataFrame with yesterday's data
    new_data = pd.DataFrame({
        'date': [yesterday_str],
        'accidents': [no_yesterdays_accidents]
    })

    # Append the new data to the existing DataFrame
    df = pd.concat([df, new_data], ignore_index=True)

    # Save the updated DataFrame back to the CSV
    df.to_csv(csv_path, index=False)
    print(f"Data for {yesterday_str} has been added to the CSV.")

# Example usage
csv_path = '../data/police_api_backfill.csv'
update_csv_with_yesterdays_accidents(csv_path)


Data for 2025-01-18 has been added to the CSV.
